<a href="https://colab.research.google.com/github/Shreyas-13/Tensorflow-Developer/blob/main/00_Tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow Fundamentals

Outline:
* Intro to tensors
* Getting info from tensors
* Manipulating tensors
* Tensors and Numpy
* Using @tf.function(a way to speed up regular python func)
* Using GPU with tensorflow

# Intro to Tensors

### Creating tensors using `tf.constant`

In [1]:
import tensorflow as tf

print(tf.__version__)

##Creating a tensor
scalar = tf.constant(7)
scalar

2.8.0


<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [2]:
#Creating a vector

vector = tf.constant([13,2], dtype=tf.float16)
vector

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([13.,  2.], dtype=float16)>

In [3]:
vector.ndim

1

In [4]:
#Creating a matrix

matrix = tf.constant([[10,20],
                      [20,30],
                     [30,40]])
matrix

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10, 20],
       [20, 30],
       [30, 40]], dtype=int32)>

In [5]:
matrix.ndim

2

In [6]:
#Creating a 3-D tensor

tensor = tf.constant([[[10,20,30],
                       [20,30,40]],
                      [[13,2,2000],
                       [13,2,2001]],
                      [[16,11,2002],
                       [12,12,12]],
                      [[12,23,43],
                      [12,45,76]]])
tensor

<tf.Tensor: shape=(4, 2, 3), dtype=int32, numpy=
array([[[  10,   20,   30],
        [  20,   30,   40]],

       [[  13,    2, 2000],
        [  13,    2, 2001]],

       [[  16,   11, 2002],
        [  12,   12,   12]],

       [[  12,   23,   43],
        [  12,   45,   76]]], dtype=int32)>

In [7]:
tensor.ndim

3

* scalar: A variable with single value
* vector: A set of values in single dimension[a single subscript is requiured hence 1-D]
* matrix: A set of vector having 2-dimension[2 subscripts required hence 2-D e.g. arr[0][0]]
* tensor: A set of matrices having n-dimension(dimension refers to the no. of digits required to indicate a particular element in an array)

### Creating tensors with `tf.Variable`

In [8]:
##Creating a tensor using tf.Variable

changable_tensor = tf.Variable([10,23])
changable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10, 23], dtype=int32)>

In [9]:
##Updating the tensor value
changable_tensor[0].assign(13)
changable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([13, 23], dtype=int32)>

Summary:
  * tf.Variable unlike tf.constant can create tensors that can be updated when required

**Note-** It is good practice to declare your tensors as constant and change them to variable when required.

### Creating Random Tensors

Random tensors are used when we're trying to initialize the weights of a NN

In [10]:
'''Creating a random tensor with with values ranging b/w -1,1 i.e. Normal
Distribution'''
tf.random.set_seed(1)

random_1 = tf.random.Generator.from_seed(42)
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

random_1 == random_2

'''We can also use Uniform Distribution if we want a value between 0,1'''

random_3 = tf.random.Generator.from_seed(9)
random_3 = random_3.uniform(shape=(1,2))
random_3

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.8203654 , 0.70470357]], dtype=float32)>

### Shuffling the tensors

Required so that the dataset is mixedd randomly and not same type of data is repeated together.

In [11]:
import numpy as np

matrix = np.random.rand(6).reshape(3,2)

not_shuffeled = tf.constant(matrix)
not_shuffeled

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[0.56345189, 0.72348053],
       [0.61771805, 0.61072883],
       [0.5019371 , 0.50079547]])>

In [12]:
'''Here even using seed doesn't guarantee same output everytime cz of global and 
operational-level seed'''

shuffeled = tf.random.shuffle(not_shuffeled, seed=1)
shuffeled

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[0.56345189, 0.72348053],
       [0.61771805, 0.61072883],
       [0.5019371 , 0.50079547]])>

In [13]:
'''Here setting tf.random.set_seed() resolves the problem'''

tf.random.set_seed(1)
shuffeled = tf.random.shuffle(not_shuffeled, seed=11)
shuffeled

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[0.61771805, 0.61072883],
       [0.5019371 , 0.50079547],
       [0.56345189, 0.72348053]])>

### Getting Information from tensors

A tensor has following attributes:
  * Shape - Shape of tensor tensor.shape
  * Rank - No of dimensions tensor.ndim
  * Axis - tensor[0] or tensor[:, 1]
  * Size - No of elements in the tensor tf.size(tensor)

In [14]:
##func to extract all these info

def info(tensor):
  print('Datatype of Elements: ', tensor.dtype)
  print('Rank: ', tensor.ndim)
  print('Shape: ', tensor.shape)
  print('Axis 0: ', tensor[0])
  print('Size: ', tf.size(tensor).numpy())

if __name__ == '__main__':
  tf.random.set_seed(100)
  rank_4_tensor = tf.random.Generator.from_seed(32)
  rank_4_tensor = rank_4_tensor.normal(shape=(2,3,4,5))
  info(rank_4_tensor)

Datatype of Elements:  <dtype: 'float32'>
Rank:  4
Shape:  (2, 3, 4, 5)
Axis 0:  tf.Tensor(
[[[ 0.7901182   1.585549    0.4356279   0.23645182 -0.1589871 ]
  [ 1.302304    0.9592239   0.85874265 -1.5181769   1.4020647 ]
  [ 1.5570306  -0.96762174  0.495291   -0.648484   -1.8700892 ]
  [ 2.7830641  -0.645002    0.18022095 -0.14656258  0.34374258]]

 [[ 0.41367555  0.17573498 -1.0871261   0.45905176  0.20386009]
  [ 0.562024   -2.3001142  -1.349454    0.81485     1.2790666 ]
  [ 0.02203509  1.5428121   0.78953624  0.53897345 -0.48535708]
  [ 0.74055266  0.31662667 -1.4391748   0.58923835 -1.4268045 ]]

 [[-0.7565803  -0.06854702  0.07595026 -1.2573844  -0.23193763]
  [-1.8107855   0.09988727 -0.50998646 -0.7535805  -0.57166284]
  [ 0.1480774  -0.23362993 -0.3522796   0.40621263 -1.0523509 ]
  [ 1.2054597   1.6874489  -0.4462975  -2.3410842   0.99009085]]], shape=(3, 4, 5), dtype=float32)
Size:  120


### Indexing Tensor

In [15]:
##Slicing in tensorflow is same as python

rank_4_tensor[:1, :2, :3, :]

<tf.Tensor: shape=(1, 2, 3, 5), dtype=float32, numpy=
array([[[[ 0.7901182 ,  1.585549  ,  0.4356279 ,  0.23645182,
          -0.1589871 ],
         [ 1.302304  ,  0.9592239 ,  0.85874265, -1.5181769 ,
           1.4020647 ],
         [ 1.5570306 , -0.96762174,  0.495291  , -0.648484  ,
          -1.8700892 ]],

        [[ 0.41367555,  0.17573498, -1.0871261 ,  0.45905176,
           0.20386009],
         [ 0.562024  , -2.3001142 , -1.349454  ,  0.81485   ,
           1.2790666 ],
         [ 0.02203509,  1.5428121 ,  0.78953624,  0.53897345,
          -0.48535708]]]], dtype=float32)>

In [16]:
##Expanding dimension of a tensor this is done to make uniform tensors of same size
'''Axis denotes the index at which one wants to add a dimension for example if 
shape of the tensor is like(2,3,4,5) and axis=1 is passed then the new shape 
will be (2,1,3,4,5)'''
rank_2_tensor = tf.random.Generator.from_seed(32)
rank_2_tensor = rank_2_tensor.normal(shape=(2,2))
print(rank_2_tensor)
print('=================')
tf.expand_dims(rank_2_tensor, axis=-1) ##axis=-1 indicates adding one extra dimension at the end


tf.Tensor(
[[0.7901182  1.585549  ]
 [0.4356279  0.23645182]], shape=(2, 2), dtype=float32)


<tf.Tensor: shape=(2, 2, 1), dtype=float32, numpy=
array([[[0.7901182 ],
        [1.585549  ]],

       [[0.4356279 ],
        [0.23645182]]], dtype=float32)>

### Manipulating tensors
**Basic Operations-**
`+`, `-`, `*`, `/`

In [17]:
##Basic Operation Addition

tensor = tf.constant([[2,3], 
                      [4,5]])
print('Add=',  tensor+10)
print('Mul=',  tensor*10)
print('Sub=',  tensor-10)
print('Div=',  tensor/10)

Add= tf.Tensor(
[[12 13]
 [14 15]], shape=(2, 2), dtype=int32)
Mul= tf.Tensor(
[[20 30]
 [40 50]], shape=(2, 2), dtype=int32)
Sub= tf.Tensor(
[[-8 -7]
 [-6 -5]], shape=(2, 2), dtype=int32)
Div= tf.Tensor(
[[0.2 0.3]
 [0.4 0.5]], shape=(2, 2), dtype=float64)


In [18]:
##We can use built-in functions as well(faster as makes use of GPU)
print('Multiply=',tf.math.multiply(tensor, 10))
print('Add=',tf.math.add(tensor, 10))
print('Div=',tf.math.divide(tensor, 10))
# print('Add=',tf.math.multiply(tensor, 10))


Multiply= tf.Tensor(
[[20 30]
 [40 50]], shape=(2, 2), dtype=int32)
Add= tf.Tensor(
[[12 13]
 [14 15]], shape=(2, 2), dtype=int32)
Div= tf.Tensor(
[[0.2 0.3]
 [0.4 0.5]], shape=(2, 2), dtype=float64)


### Matrix Multiplication

In [19]:
## Matrix Multiplication

print(tf.matmul(tensor, tensor))

## In python we use @ symbol for matrix multiplication
print(tensor @ tensor)

tf.Tensor(
[[16 21]
 [28 37]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[16 21]
 [28 37]], shape=(2, 2), dtype=int32)


In [20]:
## Other useful operations
##Transpose of a matrix

tf.random.set_seed(10)

newMat = tf.random.Generator.from_seed(32)
newMat2 = tf.random.Generator.from_seed(2)
newMat = newMat.normal(shape=(3,2))
newMat2 = newMat2.normal(shape=(3,2))

newMat, newMat2

##transposing newMat
tf.transpose(newMat)

##Reshaping a matrix
tf.reshape(newMat2, shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-0.1012345 , -0.2744976 ,  1.4204658 ],
       [ 1.2609464 , -0.43640924, -1.9633987 ]], dtype=float32)>

### Changing DataType of Tensor

In [21]:
'''Float16 works faster on GPUs as compared to float32 thus reduced precision is
required'''

B = tf.random.Generator.from_seed(10)
B = B.normal(shape=(2,2))
B

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-0.29604465, -0.21134205],
       [ 0.01063002,  1.5165398 ]], dtype=float32)>

In [22]:
'''Changing dtype to float16 using `tf.cast()`'''

B = tf.cast(B, dtype=tf.float16)
B

<tf.Tensor: shape=(2, 2), dtype=float16, numpy=
array([[-0.2961 , -0.2113 ],
       [ 0.01063,  1.517  ]], dtype=float16)>

### Tensor Aggregation
  * Max
  * Min
  * Mean
  * Sum
  * Absolute

In [23]:
tensor = tf.random.Generator.from_seed(5)
tensor = tensor.normal(shape=(3,3))
tensor

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 1.0278524 ,  0.27974114, -0.01347923],
       [ 1.845181  ,  0.97061104, -1.0242516 ],
       [-0.6544423 , -0.29738766, -1.3240396 ]], dtype=float32)>

In [24]:
## Max along axis 0 and axis 1
print(tf.reduce_max(tensor, axis=0))
print(tf.reduce_max(tensor, axis=1))


tf.Tensor([ 1.845181    0.97061104 -0.01347923], shape=(3,), dtype=float32)
tf.Tensor([ 1.0278524   1.845181   -0.29738766], shape=(3,), dtype=float32)


In [25]:
## Min along the axis 0 and axis 1
print(tf.reduce_min(tensor, axis=0))
print(tf.reduce_min(tensor, axis=1))

tf.Tensor([-0.6544423  -0.29738766 -1.3240396 ], shape=(3,), dtype=float32)
tf.Tensor([-0.01347923 -1.0242516  -1.3240396 ], shape=(3,), dtype=float32)


In [26]:
## Mean along the axes
print(tf.reduce_mean(tensor))
# print(tf.reduce_mean(tensor))

tf.Tensor(0.08997614, shape=(), dtype=float32)


In [27]:
## Sum
print(tf.reduce_sum(tensor))
# print(tf.reduce_sum(tensor, axis=1))

tf.Tensor(0.80978525, shape=(), dtype=float32)


In [29]:
print(tf.abs(tensor))

tf.Tensor(
[[1.0278524  0.27974114 0.01347923]
 [1.845181   0.97061104 1.0242516 ]
 [0.6544423  0.29738766 1.3240396 ]], shape=(3, 3), dtype=float32)


In [52]:
##Variance and Standard Deviation=> use numpy instead

import tensorflow_probability as tfp

print(tfp.stats.variance(tensor))

print(tf.math.reduce_std(tensor))

tf.Tensor([1.0829176  0.26868883 0.31434467], shape=(3,), dtype=float32)
tf.Tensor(0.9847583, shape=(), dtype=float32)


### ABC